# 1. Configuraciones

In [ ]:
# Características de la MV
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Declarar variables
HOME ="/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion"
HOME

'/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion'

# 2. Conectarse a drive

In [ ]:
# Conectarse a drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Establecer directorio de trabajo
%cd {HOME}

/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion


In [ ]:
!ls -l

total 761
-rw------- 1 root root  18968 Mar  4 16:28 '1. Data download.ipynb'
-rw------- 1 root root 295793 Mar  5 02:30 '2. Data Understand.ipynb'
-rw------- 1 root root  72065 Mar 14 02:52 '3. Image Annotation.ipynb'
-rw------- 1 root root 382388 Mar 14 02:45  annotation_videos.xlsx
drwx------ 2 root root   4096 Mar  4 15:35  data
-rw------- 1 root root    176 Mar 11 18:08 'Jam sin título.gjam'
drwx------ 2 root root   4096 Mar 12 14:20  test


# 3. Extraer imágenes de video

In [ ]:
!pip install timm
!pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=e9861766153d09df31e64506eb90c2d0bfac9936cfa32bd90a1fc86cd3365a95
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire


In [ ]:
import glob
import os
import numpy as np
import math
import pandas as pd
import torch
import cv2
import fire
from tqdm import tqdm

torch.backends.cudnn.benchmark = True

from timm.data import ImageDataset, create_loader

In [ ]:
# Función para leer y editar la data de las anotaciones en train.
def image_annotation(path_data):
    """
    Función para leer y editar la data de las anotaciones de jugadas en un video dado.
    path_data = Es la ruta del directorio dónde se encuentra el archivo csv.
    """
    # Leemos el archivo train.csv con las anotaciones.
    train = pd.read_csv(path_data)
    # Redondeamos time al segundo más cercano
    train['second'] = [int(np.round(x,0)) for x in train.time]
    # Construimos el id de la imagen
    train['image_id'] = train.video_id + '-second' + train.second.astype(str)
    # Construímos el label o etiqueta de cada imagen
    train['label0'] = np.where(train.event_attributes.isna(), np.nan, train.event) # Label = event is event != start or end else nan
    train['label1'] = train['label0'].fillna(method = 'backfill') # Reemplazamos na de labels por el valor anterior
    train['label2'] = train['label0'].fillna(method = 'ffill') # Reemplazamos na de labels por el valor posterior
    # Implementamos lógica de imputación 
    train['label'] = np.where(train.event == 'start', train.label1,
                            np.where(train.event=='end',train.label2, train.label0))
    # Eliminamos imagenes duplicadas.
    train = train.drop_duplicates(['image_id'], keep = 'first')
    # Seleccionamos columnas necesarias
    dict_labels = dict(zip(train.image_id, train.label))
    train = train.drop(['label0', 'label1', 'label2'], axis = 1)#[['image_id', 'label']].copy()
    return train, dict_labels

# Función para extraer imágenes
def extract_images(video_path, out_dir, img_size, dict_labels):
    """
    Función para extraer imágenes de un video, se extrae una imagen x segundo.
    video_path = Dirección del video del cual se quiere extraer los frames o imágenes.
    out_dir = directorio dónde se almacenarán la ímagenes extraídas del video.
    img_size = tamaño de las imágenes de salida.
    """
    video_name = os.path.basename(video_path).split('.')[0] # Extraemos la ruta dónde esta el video.
    cam = cv2.VideoCapture(video_path) # Preparamos para la captura
    fps = int(cam.get(cv2.CAP_PROP_FPS)) # Cantidad de FPS
    print(video_path, fps)
    frame_count = 0  # Contador
    while cam.isOpened():
        current_fps = int(cam.get(1)) # Frame actual
        #print(current_fps)
        successed, img = cam.read() # Leemos el frame actual
        try:
            img = cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_AREA) # Redimensianamos la imagen
        except:
            break
        # Verificamos que el frame del cual se va a generar la imagen, corresponde al segundo siguiente.
        if (current_fps % math.floor(fps)==0):
            image_id = f'{video_name}-second{int(current_fps/fps)}'
            if image_id in dict_labels.keys():
                outfile = f'{out_dir}/{dict_labels[image_id]}/{image_id}.jpg' # Archivos de salida
            else:
                outfile = f'{out_dir}/background/{image_id}.jpg' # Archivos de salida
            
            cv2.imwrite(outfile, img) # Guardamos la imagen
        if not successed:
            break # Si ya no hay más frames paramos
        frame_count += 1

################### Función para implementar todo el proceso

def main():
    # Leer anotaciones
    path_data = f'{HOME}/data/train.csv'
    train, dict_labels = image_annotation(path_data)
    # Data de las anotaciones
    train.to_excel(f'{HOME}/annotation_videos.xlsx', index = False)

    # Lista de videos de entrenamiento
    train_videos = sorted(glob.glob('./data/train/*'))
    # Recorremos toda la lista de videos
    for video_path in train_videos:
        #print(video_path)
        out_dir = './images'
        img_size = (544,544)
        # Extraer imágenes y almacenarlas dónde le correponda.
        extract_images(video_path, out_dir, img_size, dict_labels)

    print("El proceso ha terminado con éxito.")
    

In [ ]:
# Construimos el directorio dónde se almacenarán las imágenes
!mkdir -p images/background images/challenge images/play images/throwin

In [ ]:
%%time
########### FUNCTION - MAIN
if __name__ == '__main__':
    fire.Fire(main)

./data/train/1606b0e6_0.mp4 25
./data/train/1606b0e6_1.mp4 25
./data/train/35bd9041_0.mp4 25
./data/train/35bd9041_1.mp4 25
./data/train/3c993bd2_0.mp4 25
./data/train/3c993bd2_1.mp4 25
./data/train/407c5a9e_1.mp4 25
./data/train/4ffd5986_0.mp4 25
./data/train/9a97dae4_1.mp4 25
./data/train/cfbe2e94_0.mp4 25
./data/train/cfbe2e94_1.mp4 25
./data/train/ecf251d4_0.mp4 25
El proceso ha terminado con éxito.


ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: ignored